In [1]:
#ignore warnings
import warnings
warnings.filterwarnings("ignore")

#wrangling
import pandas as pd
import numpy as np

#explore
import scipy.stats as stats

#visuals
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import env
import acquire
import prepare
# set pandas options
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)


#default pandas decimal display formatting
pd.options.display.float_format='{:20,.2f}'.format

# Acquire.py

## 1 acquire & summarize

- Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.
        
- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
        
- Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
        
- Only include properties that include a latitude and longitude value

In [2]:
zillow = acquire.get_zillow_data()

## 2 Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [3]:
zillow.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,logerror,transactiondate,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,nan,nan,261.00,nan,nan,nan,nan,14297519,0.03,2017-01-01,nan,3.50,4.00,nan,3.50,nan,nan,"3,100.00","3,100.00",nan,nan,nan,nan,"6,059.00",nan,3.00,2.00,633.00,nan,"33,634,931.00","-117,869,207.00","4,506.00",nan,nan,nan,nan,nan,122,None,"60,590,630.07","53,571.00","1,286.00",nan,"96,978.00",0.00,1.00,nan,nan,nan,"1,998.00",nan,nan,"485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72",None,nan,"60,590,630,072,012.00",None,None,None,None,Single Family Residential,None,None
1,nan,nan,261.00,nan,nan,nan,nan,17052889,0.06,2017-01-01,nan,1.00,2.00,nan,1.00,nan,"1,465.00","1,465.00","1,465.00",nan,nan,"1,465.00",nan,"6,111.00",1.00,1.00,1.00,0.00,nan,"34,449,266.00","-119,281,531.00","12,647.00",nan,nan,nan,nan,nan,1110,None,"61,110,010.02","13,091.00","2,061.00",nan,"97,099.00",5.00,nan,nan,nan,nan,"1,967.00",1.00,nan,"88,000.00","464,000.00","2,016.00","376,000.00","5,672.48",None,nan,"61,110,010,023,006.00",None,None,None,None,Single Family Residential,None,None
2,nan,nan,261.00,nan,nan,nan,nan,14186244,0.01,2017-01-01,nan,2.00,3.00,nan,2.00,nan,nan,"1,243.00","1,243.00",nan,nan,nan,nan,"6,059.00",nan,2.00,2.00,440.00,nan,"33,886,168.00","-117,823,170.00","8,432.00",1.00,nan,nan,nan,1.00,122,None,"60,590,218.02","21,412.00","1,286.00",nan,"97,078.00",6.00,nan,nan,nan,nan,"1,962.00",1.00,nan,"85,289.00","564,778.00","2,016.00","479,489.00","6,488.30",None,nan,"60,590,218,022,012.00",None,None,None,None,Single Family Residential,None,None
3,nan,nan,261.00,2.00,nan,nan,nan,12177905,-0.10,2017-01-01,nan,3.00,4.00,8.00,3.00,nan,nan,"2,376.00","2,376.00",nan,nan,nan,nan,"6,037.00",nan,3.00,nan,nan,nan,"34,245,180.00","-118,240,722.00","13,038.00",1.00,nan,nan,nan,1.00,0101,LCR110000*,"60,373,001.00","396,551.00","3,101.00",nan,"96,330.00",0.00,nan,1.00,nan,nan,"1,970.00",nan,nan,"108,918.00","145,143.00","2,016.00","36,225.00","1,777.51",None,nan,"60,373,001,001,006.00",None,None,None,Central,Single Family Residential,None,None
4,nan,nan,266.00,2.00,nan,nan,1.00,10887214,0.01,2017-01-01,nan,3.00,3.00,8.00,3.00,nan,nan,"1,312.00","1,312.00",nan,nan,nan,nan,"6,037.00",nan,3.00,nan,nan,nan,"34,185,120.00","-118,414,640.00","278,581.00",1.00,nan,nan,nan,1.00,010C,LAR3,"60,371,236.01","12,447.00","3,101.00","268,548.00","96,451.00",0.00,nan,1.00,nan,nan,"1,964.00",nan,nan,"73,681.00","119,407.00","2,016.00","45,726.00","1,533.89",None,nan,"60,371,236,012,000.00",Central,None,None,Central,Condominium,None,None


In [4]:
zillow.shape

(77381, 67)

## 3 Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values. 

In [5]:
# lets count and sum all the nulls in the zillow database
total_missing = zillow.isnull().sum()
total_missing.head()

typeconstructiontypeid    77159
storytypeid               77331
propertylandusetypeid         0
heatingorsystemtypeid     27941
buildingclasstypeid       77366
dtype: int64

In [6]:
# creating variable call avg_missing which takes total_missing and divides it by the number of rows in the zillow
avg_missing = total_missing / zillow.shape[0]
avg_missing.head()

typeconstructiontypeid                   1.00
storytypeid                              1.00
propertylandusetypeid                    0.00
heatingorsystemtypeid                    0.36
buildingclasstypeid                      1.00
dtype: float64

In [7]:
# created a new pandas.DataFrame out of the previous filtering we did
missing_columns = pd.DataFrame({'num_rows_missing': total_missing, 'pct_rows_missing': avg_missing})
missing_columns.head()

,num_rows_missing,pct_rows_missing
typeconstructiontypeid,77159,1.00
storytypeid,77331,1.00
propertylandusetypeid,0,0.00
heatingorsystemtypeid,27941,0.36
buildingclasstypeid,77366,1.00


In [8]:
# put it all together to make a function


# def nulls_by_columns(df):
#     total_missing = zillow.isnull().sum()
#     avg_missing = total_missing / zillow.shape[0]
#     missing_columns = pd.DataFrame({'num_rows_missing': total_missing, 'pct_rows_missing': avg_missing})
#     return missing_columns

# Prepare.py

## 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). 
- There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. 
- But for this exercise, do not purely filter by unitcnt as we did previously. 
- Add some new logic that will reduce the number of properties that are falsely removed. 
- You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined. 

In [9]:
# shows all the unique columns so we can identify items to filter by
zillow.propertylandusedesc.value_counts()

Single Family Residential                     52320
Condominium                                   19294
Duplex (2 Units, Any Combination)              2009
Planned Unit Development                       1944
Quadruplex (4 Units, Any Combination)           727
Triplex (3 Units, Any Combination)              535
Cluster Home                                    333
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       58
Residential General                              37
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: propertylandusedesc, dtype: int64

In [10]:
# lets filter this data 
# data has been filtered by using the following function
zillow = prepare.filter_single_unit_properties(zillow)
print(zillow.shape)
zillow.head(3)

(54322, 67)


,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,logerror,transactiondate,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,nan,nan,261.00,nan,nan,nan,nan,14297519,0.03,2017-01-01,nan,3.50,4.00,nan,3.50,nan,nan,"3,100.00","3,100.00",nan,nan,nan,nan,"6,059.00",nan,3.00,2.00,633.00,nan,"33,634,931.00","-117,869,207.00","4,506.00",nan,nan,nan,nan,nan,122,None,"60,590,630.07","53,571.00","1,286.00",nan,"96,978.00",0.00,1.00,nan,nan,nan,"1,998.00",nan,nan,"485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72",None,nan,"60,590,630,072,012.00",None,None,None,None,Single Family Residential,None,None
1,nan,nan,261.00,nan,nan,nan,nan,17052889,0.06,2017-01-01,nan,1.00,2.00,nan,1.00,nan,"1,465.00","1,465.00","1,465.00",nan,nan,"1,465.00",nan,"6,111.00",1.00,1.00,1.00,0.00,nan,"34,449,266.00","-119,281,531.00","12,647.00",nan,nan,nan,nan,nan,1110,None,"61,110,010.02","13,091.00","2,061.00",nan,"97,099.00",5.00,nan,nan,nan,nan,"1,967.00",1.00,nan,"88,000.00","464,000.00","2,016.00","376,000.00","5,672.48",None,nan,"61,110,010,023,006.00",None,None,None,None,Single Family Residential,None,None
2,nan,nan,261.00,nan,nan,nan,nan,14186244,0.01,2017-01-01,nan,2.00,3.00,nan,2.00,nan,nan,"1,243.00","1,243.00",nan,nan,nan,nan,"6,059.00",nan,2.00,2.00,440.00,nan,"33,886,168.00","-117,823,170.00","8,432.00",1.00,nan,nan,nan,1.00,122,None,"60,590,218.02","21,412.00","1,286.00",nan,"97,078.00",6.00,nan,nan,nan,nan,"1,962.00",1.00,nan,"85,289.00","564,778.00","2,016.00","479,489.00","6,488.30",None,nan,"60,590,218,022,012.00",None,None,None,None,Single Family Residential,None,None


## 2 Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

   - The input:
        - A dataframe
        
        - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
        
        - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
   - The output:
        - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
   - hint:
        - Look up the dropna documentation.
        - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
        - Make use of inplace, i.e. inplace=True/False.



In [11]:
# let's break down this function
zillow = prepare.handle_missing_values(zillow)
print(zillow.shape)
zillow.head()  

(54226, 33)


,propertylandusetypeid,heatingorsystemtypeid,parcelid,logerror,transactiondate,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,heatingorsystemdesc,propertylandusedesc
0,261.00,nan,14297519,0.03,2017-01-01,3.50,4.00,nan,3.50,"3,100.00","3,100.00","6,059.00",3.00,"33,634,931.00","-117,869,207.00","4,506.00",122,None,"60,590,630.07","53,571.00","1,286.00","96,978.00",0.00,nan,"1,998.00","485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72","60,590,630,072,012.00",None,Single Family Residential
1,261.00,nan,17052889,0.06,2017-01-01,1.00,2.00,nan,1.00,"1,465.00","1,465.00","6,111.00",1.00,"34,449,266.00","-119,281,531.00","12,647.00",1110,None,"61,110,010.02","13,091.00","2,061.00","97,099.00",5.00,nan,"1,967.00","88,000.00","464,000.00","2,016.00","376,000.00","5,672.48","61,110,010,023,006.00",None,Single Family Residential
2,261.00,nan,14186244,0.01,2017-01-01,2.00,3.00,nan,2.00,"1,243.00","1,243.00","6,059.00",2.00,"33,886,168.00","-117,823,170.00","8,432.00",122,None,"60,590,218.02","21,412.00","1,286.00","97,078.00",6.00,nan,"1,962.00","85,289.00","564,778.00","2,016.00","479,489.00","6,488.30","60,590,218,022,012.00",None,Single Family Residential
3,261.00,2.00,12177905,-0.10,2017-01-01,3.00,4.00,8.00,3.00,"2,376.00","2,376.00","6,037.00",3.00,"34,245,180.00","-118,240,722.00","13,038.00",0101,LCR110000*,"60,373,001.00","396,551.00","3,101.00","96,330.00",0.00,1.00,"1,970.00","108,918.00","145,143.00","2,016.00","36,225.00","1,777.51","60,373,001,001,006.00",Central,Single Family Residential
6,261.00,2.00,12095076,-0.00,2017-01-01,3.00,4.00,9.00,3.00,"2,962.00","2,962.00","6,037.00",3.00,"34,145,202.00","-118,179,824.00","63,000.00",0101,PSR2,"60,374,608.00","47,019.00","3,101.00","96,293.00",0.00,1.00,"1,950.00","276,684.00","773,303.00","2,016.00","496,619.00","9,516.26","60,374,608,001,014.00",Central,Single Family Residential


In [12]:
# next